<a href="https://colab.research.google.com/github/SukhandeepKaurShergill/Transfer-Learning/blob/main/Copy_of_data_prepration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import os
from PIL import Image
import numpy as np
from scipy import ndimage
import imageio
import matplotlib.image as mpimage
import cv2
import glob
import h5py
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import imageio


In [2]:
from google.colab import files
uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_path="/content/drive/MyDrive/train"
pixel_depth=255.0
train_labels=os.listdir(train_path)
#test_labels=os.listdir(test_path)
train_labels.sort()
#test_labels.sort()
print(train_labels)
nb_classes= 2
global_features_train=[]
train_classes=[]
#test_classes=[]


i,j=0,0
k=0


['cat', 'dog']


In [4]:
for training_name in train_labels:
    path=os.path.join(train_path,training_name,'*')
    files=glob.glob(path)
    current_label=training_name
    k=1
    for fl in files:
        image=(imageio.imread(fl).astype(float)-pixel_depth/2)/pixel_depth
        image=cv2.resize(image, (224, 224))
        global_feature=np.hstack([image])
        train_classes.append(current_label)
        global_features_train.append(global_feature)

        i+=1
        k+1
    print("[status] processed folder: {}".format(current_label))
    j+=1
print("[status] completed global feature extraction..")
print("[status] feature vector size {}".format(np.array(global_features_train).shape))
print("[status] training labels {}".format(np.array(train_classes).shape))


[status] processed folder: cat
[status] processed folder: dog
[status] completed global feature extraction..
[status] feature vector size (2936, 224, 224, 3)
[status] training labels (2936,)


In [5]:
#labels = (np.arange(nb_classes) == labels[:,None]).astype(np.float32)
targetNames=np.unique(train_classes)
le=LabelEncoder()
target=le.fit_transform(train_classes)
print("[status] training labels encoded...")

[status] training labels encoded...


In [ ]:
#n_samples, nx, ny=np.array(global_features_train).shape
#d2_global_features=np.array(global_features_train).reshape((n_samples, nx*ny))
#scaler=MinMaxScaler(feature_range=(0,1))
#rescaled_features=scaler.fit_transform(d2_global_features)

In [6]:


print("[Status] target label{}".format(target))
print("[Status] target label shape {}".format(target.shape))


[Status] target label[0 0 0 ... 1 1 1]
[Status] target label shape (2936,)


In [7]:
h5f_data=h5py.File('cat_dog_data.h5','w')
h5f_data.create_dataset('dataset_1',data=np.array(global_features_train))

h5f_label=h5py.File('cat_dog_labels.h5','w')
h5f_label.create_dataset('dataset_1',data=np.array(target))


<HDF5 dataset "dataset_1": shape (2936,), type "<i8">

In [8]:
h5f_data.close()
h5f_label.close()

In [9]:
batch_size = 30
num_classes = 2
epochs = 30

In [10]:
h5f_data = h5py.File('/content/cat_dog_data.h5', 'r')
h5f_label = h5py.File('/content/cat_dog_labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

In [11]:
# verify the shape of the feature vector and labels
print ("[STATUS] features shape: {}".format(global_features.shape))
print ("[STATUS] labels shape: {}".format(global_labels.shape))

print ("[STATUS] training started...")

[STATUS] features shape: (2936, 224, 224, 3)
[STATUS] labels shape: (2936,)
[STATUS] training started...


In [ ]:
from sklearn.model_selection import train_test_split
results = []
names = []
scoring = "accuracy"
seed=9
(trainData, validData, trainLabels, validLabels) = train_test_split(np.array(global_features),
                                                                                          np.array(global_labels),
                                                                                          test_size=.1,
                                                                                          random_state=seed)